<a href="https://colab.research.google.com/github/mlwithshri/cat-vs-dog-classifier/blob/main/dogs_and_cats_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this is cat vs dog classifier cnn based project

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dogs-vs-cats


In [ ]:
import os

data_dir = "/kaggle/input/dogs-vs-cats"
print(os.listdir(data_dir))


['dogs_vs_cats', 'test', 'train']


In [ ]:
print(os.listdir(os.path.join(data_dir, "train"))[:10])  # first 10 files
print(os.listdir(os.path.join(data_dir, "test"))[:10])   # first 10 files


['dogs', 'cats']
['dogs', 'cats']


new model


In [ ]:
from google.colab import files
files.download("cats_vs_dogs_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# new model

In [ ]:
# 1. Imports
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 2. Paths
data_dir = "/kaggle/input/dogs-vs-cats"   # dataset root
img_size = (128, 128)
batch_size = 32

# 3. Load datasets
train_ds = keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, "train"),
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    label_mode="int"   # ensures integer labels
)

val_ds = keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, "test"),
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    label_mode="int"
)

# Check class names
class_names = train_ds.class_names
print("Class names:", class_names)  # should be ['cat', 'dog']

# 4. Normalize & cache datasets
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# 5. Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# 6. Build CNN Model
model = keras.Sequential([
    data_augmentation,
    layers.Conv2D(32, (3,3), activation="relu", input_shape=img_size+(3,)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  # binary classification
])

# 7. Compile
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# 8. Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# 9. Save model
model.save("cats_vs_dogs_model1.keras")

# 10. Prediction function
def predict_image(model, img_path, img_size=(128,128)):
    img = keras.utils.load_img(img_path, target_size=img_size)
    img_array = keras.utils.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)  # (1, h, w, 3)

    prob = model.predict(img_array)[0][0]  # probability of dog
    if prob > 0.5:
        return "dog", prob
    else:
        return "cat", 1 - prob



Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Class names: ['cats', 'dogs']
Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 43ms/step - accuracy: 0.5485 - loss: 0.6953 - val_accuracy: 0.6790 - val_loss: 0.6313
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.6564 - loss: 0.6229 - val_accuracy: 0.7116 - val_loss: 0.5651
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.7211 - loss: 0.5545 - val_accuracy: 0.7600 - val_loss: 0.5185
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.7511 - loss: 0.5063 - val_accuracy: 0.7920 - val_loss: 0.4411
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.7736 - loss: 0.4724 - val_accuracy: 0.8028 - val_loss: 0.4287
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.8011 - loss: 0.4352 - val_accuracy: 0.8278 - val_loss: 0.3802
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.8107 - loss: 0.4237 - val_accuracy: 0.8226 - val_loss: 0.3902
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.8271 - loss: 0.3937 - val_accurac

In [ ]:
history2 = model.fit(train_ds, validation_data=test_ds, epochs=5)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
img_path = "/content/cat.54.jpg"   # make sure path is correct
pred, conf = predict_image(model, img_path)
print(f"Prediction: {pred} (confidence: {conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
Prediction: cat (confidence: 0.93)


In [ ]:
import os

train_dir = os.path.join("/kaggle/input/dogs-vs-cats", "train")
cat_count = len(os.listdir(os.path.join(train_dir, "cats")))
dog_count = len(os.listdir(os.path.join(train_dir, "dogs")))

print("Cats:", cat_count, "Dogs:", dog_count)

Cats: 10000 Dogs: 10000


In [ ]:
img_path = "/content/dog.22.jpg"   # make sure path is correct
pred, conf = predict_image(model, img_path)
print(f"Prediction: {pred} (confidence: {conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: dog (confidence: 1.00)


In [ ]:
test_loss, test_acc = model.evaluate(val_ds)
print(f"Test Accuracy: {test_acc:.2f}")
print(f"Test Loss: {test_loss:.4f}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8687 - loss: 0.3145
Test Accuracy: 0.86
Test Loss: 0.3376


In [ ]:
img_path = "/content/dog.38.jpg"   # make sure path is correct
pred, conf = predict_image(model, img_path)
print(f"Prediction: {pred} (confidence: {conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: dog (confidence: 1.00)


In [ ]:
img_path = "/content/cat.158.jpg"
pred, conf = predict_image(model, img_path)
print(f"Prediction: {pred} (confidence: {conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: cat (confidence: 0.51)


In [ ]:
img_path = "/content/cat.135.jpg"   # make sure path is correct
pred, conf = predict_image(model, img_path)
print(f"Prediction: {pred} (confidence: {conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: cat (confidence: 0.53)


In [ ]:
from google.colab import files
files.download("cats_vs_dogs_model2.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>